In [2]:
#notebook based on zflemings:https://nbviewer.jupyter.org/github/zflamig/dask-era5/blob/main/notebook/era5_fargate_dask.ipynb
import xarray as xr
import fsspec
import dask
import s3fs
import numpy as np
xr.set_options(display_style="html")  #display dataset nicely 

In [3]:
url = 's3://era5-pds/1980/01/data/sea_surface_temperature.nc'
ncfile = fsspec.open(url)
ds = xr.open_dataset(ncfile.open())
ds = ds.mean({'time0'})

In [4]:
ds

<xarray.Dataset>
Dimensions:                  (lat: 721, lon: 1440)
Coordinates:
  * lon                      (lon) float32 0.0 0.25 0.5 ... 359.25 359.5 359.75
  * lat                      (lat) float32 90.0 89.75 89.5 ... -89.75 -90.0
Data variables:
    sea_surface_temperature  (lat, lon) float32 271.4375 271.4375 ... nan nan

In [11]:
years,months=[],[]
[years.append(str(y)) for y in np.arange(1979,2020)]
[months.append(str(m).zfill(2)) for m in np.arange(1,13)]

file_pattern = 's3://era5-pds/{year}/{month}/data/sea_surface_temperature.nc' #air_temperature_at_2_metres.nc'
fnames = [file_pattern.format(year=year,month=month) for year in years for month in months]


In [3]:
years,months=[],[]
[years.append(str(y)) for y in np.arange(1979,2020)]
[months.append(str(m).zfill(2)) for m in np.arange(1,13)]

file_pattern = 's3://era5-pds/{year}/{month}/data/sea_surface_temperature.nc' #air_temperature_at_2_metres.nc'

@dask.delayed
def s3open(path):
    fs = s3fs.S3FileSystem(anon=True, default_fill_cache=False)
    return fs.open(path)

files_mapper = [s3open(file_pattern.format(year=year,month=month)) for year in years for month in months]

In [8]:
ds_av=[]
for fname in files_mapper:
    ds = xr.open_dataset(fname,engine='h5netcdf')
    ds = ds.mean('time0')
    ds_av.append(ds)

TypeError: Truth of Delayed objects is not supported

In [5]:
ds = xr.open_mfdataset(files_mapper, engine='h5netcdf', chunks={'lon':200,'lat':200,'time0':720}, concat_dim='time1', combine='nested', coords='minimal', compat='override', parallel=True)

ds

/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/core/indexing.py:1361: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/core/indexing.py:1361: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/srv/conda/envs/note

,Array,Chunk
Bytes,734.35 TB,234.24 MB
Shape,"(492, 359400, 721, 1440)","(1, 1464, 200, 200)"
Count,13757510 Tasks,4534272 Chunks
Type,float32,numpy.ndarray


In [9]:
ds

,Array,Chunk
Bytes,734.35 TB,234.24 MB
Shape,"(492, 359400, 721, 1440)","(1, 1464, 200, 200)"
Count,13757510 Tasks,4534272 Chunks
Type,float32,numpy.ndarray
